In [1]:
"""
Created on Fri Jul  5 11:39:29 2024

title: Full data analysis
description: Implementation of the solution Semi-supervised approach for segmenting tumours in MRI Images.

@author: wickramw
@github: https://github.com/bimec/image-AZ/tree/deployment
"""

'\nCreated on Fri Jul  5 11:39:29 2024\n\ntitle: Full data analysis\ndescription: Implementation of the solution Semi-supervised approach for segmenting tumours in MRI Images.\n\n@author: wickramw\n@github: https://github.com/bimec/image-AZ/tree/deployment\n'

In [2]:
# =============================================================================
import nibabel as nib
import cv2
import os
import numpy as np
import csv
from pathlib import Path

# Algorithms
print("Importing..!")
from Algorithms.ROIProcessing import ROIProcessing
from Algorithms.TumourSegmenting import segmentTumour

model_path = Path("./Algorithms/MRI_seg_best.pt")
model = segmentTumour(model_path)
print("Algorithms and Models Imported!")

Importing..!
Algorithms and Models Imported!


In [3]:
# =============================================================================
# Set the location for the dataset
dataset = Path('./dataset/train/21_post_15min_rare.nii.gz')                  # MRI images to predict tumours
dataset_labelled = Path('./dataset/train_labels/21_post_15min_rare.nii.gz')  # Groud truth label to validate the algorithm

In [4]:
# =============================================================================
# MRI
# f_mri  = r"C:/Users/wickramw/OneDrive - London South Bank University/Imaging-AZ/image-AZ-git/image-AZ/dataset/train/21_baseline_rare.nii.gz"
# f_mri = r"{}".format(input("Enter the path for image: "))
mri_data = nib.load(dataset)
mri_data = mri_data.get_fdata()


# Masks
# f_mask = r"C:/Users/wickramw/OneDrive - London South Bank University/Imaging-AZ/image-AZ-git/image-AZ/dataset/train_labels/21_baseline_rare.nii.gz"
# f_mask = r"{}".format(input("Enter the path for label: "))
# f_mask = f_mask = f_mri.split("train")[0] + "train_labels" +f_mri.split("train")[1]
mask_data = nib.load(dataset_labelled)
mask_data = mask_data.get_fdata()

print("MRI and Mask data loaded!")

MRI and Mask data loaded!


In [5]:
# =============================================================================
# Saving Raw and Mask Images
save_folder = Path('.')
save_folder_output = Path(save_folder, 'output')

try:
    save_folder_output.mkdir()
except:
    pass

# save MRI images
try:
    save_folder_output_rawMri = Path(save_folder_output, 'rawMRI')
    save_folder_output_rawMri.mkdir()
    
    for i in range(mri_data.shape[-1]):
        img_name = Path(save_folder_output_rawMri, 'rawImg_Z{}.png'.format(i))
    
        img_save = mri_data[:,:,i]/mri_data[:,:,i].max()*255 if(mri_data[:,:,i].max()>0) else mri_data[:,:,i]

        img_save = img_save.astype(np.uint8)
        cv2.imwrite(img_name, img_save)
        
    print("Raw Images Saved!")
except Exception as e:
    print(e)

# save Mask images
try:
    save_folder_output_rawMask = Path(save_folder_output, 'rawMask')
    save_folder_output_rawMask.mkdir()
    
    for i in range(mask_data.shape[-1]):
        img_name = Path(save_folder_output_rawMask, 'maskImg_Z{}.png'.format(i))
        img_save = mask_data[:,:,i]/mask_data[:,:,i].max()*255 if(mask_data[:,:,i].max()>0) else mask_data[:,:,i]
        img_save = img_save.astype(np.uint8)
        cv2.imwrite(img_name, img_save)
        
    print("Mask Images Saved!")
except Exception as e:
    print(e)

Raw Images Saved!
Mask Images Saved!


In [6]:
# =============================================================================
# Calculating ROIs
roiProcessor = ROIProcessing()
roi_data = np.zeros_like(mri_data)
for i in range(mri_data.shape[-1]):
    img = roiProcessor.prepImage(mri_data[:,:,i])    # Preprocess
    cluster_img = roiProcessor.applyClustering(img)  # Clustering 
    img_c = roiProcessor.enhanceROI(cluster_img)     # Enhance ROI
    roi_mask = roiProcessor.getROIMask(img_c)        # Get ROI Mask
    img_ROI = roiProcessor.applyMask(roi_mask)       # Get ROI Image   
    roi_data[:,:,i] = img_ROI
print("ROIs Calculated!")

ROIs Calculated!


In [7]:
# =============================================================================
# save ROI images
save_folder = Path('.')
save_folder_output = Path(save_folder, 'output')

try:
    save_folder_output_roiMri = Path(save_folder_output, 'roiMRI')
    save_folder_output_roiMri.mkdir()
    
    for i in range(roi_data.shape[-1]):
        img_name = Path(save_folder_output_roiMri, 'roiImg_Z{}.png'.format(i))
        img_save = roi_data[:,:,i]/roi_data[:,:,i].max()*255 if(roi_data[:,:,i].max() != 0) else roi_data[:,:,i]
        img_save = img_save.astype(np.uint8)
        cv2.imwrite(img_name, img_save)

    print("ROIs Images Saved!")
except Exception as e:
    print(e)

ROIs Images Saved!


In [8]:
# =============================================================================
# Tumour Segmentation
# load model
model = segmentTumour(model_path)

x_input = []
for i in range(roi_data.shape[-1]):
    image = cv2.cvtColor(roi_data[:,:,i].astype(np.uint8), cv2.COLOR_GRAY2RGB)
    x_input.append(image)
results = model.predTumour(x_input)
print("Tumour Search Completed!")


0: 320x320 (no detections), 127.8ms
1: 320x320 (no detections), 127.8ms
2: 320x320 (no detections), 127.8ms
3: 320x320 (no detections), 127.8ms
4: 320x320 1 tumour-iQcJ, 127.8ms
5: 320x320 2 tumour-iQcJs, 127.8ms
6: 320x320 2 tumour-iQcJs, 127.8ms
7: 320x320 1 tumour-iQcJ, 127.8ms
8: 320x320 1 tumour-iQcJ, 127.8ms
9: 320x320 1 tumour-iQcJ, 127.8ms
Speed: 0.5ms preprocess, 127.8ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 320)
Tumour Search Completed!


In [9]:
# Segmentation
summary = model.segmentSummary()
       
# DICE Score
dice = model.diceScore(mask_data)

# Tumour plots
predImg = model.tumourPlots()

# Tumour features
tfeatures = model.tumourFeatures()

print("Summary generated!")

Summary generated!


In [10]:
# =============================================================================
## results summary
# summary csv   : imgName, box, class, confidence, class name, segments
# dice_score
# image: pred with tumour

save_folder = Path('.')
save_folder_output = Path(save_folder, 'output')

try:
    # CSVs
    save_folder_output_resultsMri = Path(save_folder_output, 'resultsMRI')
    save_folder_output_resultsMri.mkdir()
    
    # summary csv
    keys = summary[0].keys()
    with open(Path(save_folder_output_resultsMri, 'summary.csv'), 'w', newline='') as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(summary)
    
    # dice_score
    keys = dice[0].keys()
    with open(Path(save_folder_output_resultsMri, 'dice_score.csv'), 'w', newline='') as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(dice)

    # Tumour features
    keys = tfeatures[0].keys()
    with open(Path(save_folder_output_resultsMri, 'tfeatures.csv'), 'w', newline='') as f:
        dict_writer = csv.DictWriter(f, keys)
        dict_writer.writeheader()
        dict_writer.writerows(tfeatures)
    
    # # Segmentation images
    save_folder_output_resultspredImg = Path(save_folder_output, 'predImg')
    save_folder_output_resultspredImg.mkdir()
    
    # predImages
    for i in range(len(predImg)):
        img_name = Path(save_folder_output_resultspredImg, 'predImg_Z{}.png'.format(i))
        cv2.imwrite(img_name, predImg[i]['predImg'])

    print("Prediction and Features Saved!")
except Exception as e:
    print(e)

Prediction and Features Saved!
